# Car Price prediction
Предсказание стоимости автомобиля на разных типах данных с использованием различных моделей машинного обучения и нейросетей



# Часть 1
* Наивная модель
* EDA
* Feature engineering
* CatBoost

In [1]:
!pip install -q tensorflow==2.3

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
#аугментации изображений
!pip install albumentations -q

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
!pip install pymystem3


You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [ ]:
!pip install pymorphy2

# Импортируем библиотеки

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys
import PIL
import cv2
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import f_classif
from catboost import Pool

# # keras
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import albumentations

# plt
import matplotlib.pyplot as plt
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind

import warnings
warnings.filterwarnings('ignore')

# For NLP
import nltk
from nltk.probability import FreqDist
from nltk import word_tokenize
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
import re
from collections import defaultdict
from pymorphy2 import MorphAnalyzer

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)

Python       : 3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) 
Numpy        : 1.18.5
Tensorflow   : 2.3.0


# Функции

In [6]:
#метрика МАРЕ
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# EDA
def desription(data, col, boundaries = None):
    print('Столбец', col)
    print('Распределение признака:')
    fig, ax = plt.subplots()
    if data.loc[:, col].dtypes == np.dtype('O'): # Проверка на строковый тип данных
        sns.countplot(data.loc[:, col], ax=ax)
        plt.show()
    else:
        
        data.loc[:, col].hist(ax=ax)
        ax.set_xlabel(col)
        ax.set_ylabel('count')
        plt.show()
        if boundaries is not None: # Проверка на наличие выбросов по информации из датасета
            if data.loc[(~data.loc[:, col].between(boundaries[0], boundaries[1]))&
                       pd.notnull(data.loc[:, col])].shape[0] > 0:
                print('Найдены выбросы:')
                display(data.loc[(~data.loc[:, col].between(boundaries[0], boundaries[1]))&
                       pd.notnull(data.loc[:, col])])
                
    print('Процент пропусков:', str(round(100 - data.loc[:, col].count()/data.shape[0]*100, 2)) + '%')
    print()
    print('Детальное описание:')
    print(data.loc[:, col].describe())
    print()
    print('Значения признака:')
    print(data.loc[:, col].value_counts(dropna=False, normalize=True))
    

In [7]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [ ]:
!pip freeze > requirements.txt

# DATA

Посмотрим на типы признаков:

* bodyType - категориальный
* brand - категориальный
* color - категориальный
* description - текстовый
* engineDisplacement - числовой, представленный как текст
* enginePower - числовой, представленный как текст
* fuelType - категориальный
* mileage - числовой
* modelDate - числовой
* model_info - категориальный
* name - категориальный, желательно сократить размерность
* numberOfDoors - категориальный
* price - числовой, целевой
* productionDate - числовой
* sell_id - изображение (файл доступен по адресу, основанному на sell_id)
* vehicleConfiguration - не используется (комбинация других столбцов)
* vehicleTransmission - категориальный
* Владельцы - категориальный
* Владение - числовой, представленный как текст
* ПТС - категориальный
* Привод - категориальный
* Руль - категориальный

In [8]:
DATA_DIR = '../input/sf-dst-car-price-prediction-part2/'
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели и году выпуска. 
C ней будем сравнивать другие модели.



In [ ]:
# split данных
data_train, data_test = train_test_split(train, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
# Наивная модель
predicts = []
for index, row in pd.DataFrame(data_test[['model_info', 'productionDate']]).iterrows():
    query = f"model_info == '{row[0]}' and productionDate == '{row[1]}'"
    predicts.append(data_train.query(query)['price'].median())

# заполним не найденные совпадения
predicts = pd.DataFrame(predicts)
predicts = predicts.fillna(predicts.median())

# округлим
predicts = (predicts // 1000) * 1000

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(data_test['price'], predicts.values[:, 0]))*100:0.2f}%")

# EDA

Проведем быстрый анализ данных для того, чтобы понимать, сможет ли с этими данными работать наш алгоритм.

In [ ]:
train.info()

пропуски в 2х признаках: Владельцы и Владение

In [ ]:
test.info()

пропуски только в признаке Владельцы

In [ ]:
set(train.columns)- set(test.columns)

В test отсутствует только таргет price', что вполне логично

In [27]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
test['price'] = 0 # в тесте у нас нет значения price, мы его должны предсказать, поэтому пока просто заполняем нулями

data = test.append(train, sort=False).reset_index(drop=True) # объединяем
print(train.shape, test.shape, data.shape)

(6682, 23) (1671, 23) (8353, 23)


## Обработка фичей

In [ ]:
desription(data, 'bodyType')

Пропусков нет. Это категориальный признак, поэтому можно использовать его для CatBoost как есть, или сделать One-Hot Encoding для нейросетей. Пригодится для NLP (дальше посмотрим, как его преобразовать или куда добавить)

In [ ]:
desription(data, 'brand')

Пропусков нет. Всего три варианта марки автомобиля. Это категориальный признак, поэтому можно использовать его для CatBoost как есть, или сделать One-Hot Encoding для нейросетей. Пригодится для NLP (дальше посмотрим, как его преобразовать или куда добавить)

In [ ]:
desription(data, 'color')

Пропусков нет. Более 41% автомобилей  черные. Это категориальный признак, поэтому можно использовать его для CatBoost как есть, или сделать One-Hot Encoding для нейросетей. Пригодится для NLP (дальше посмотрим, как его преобразовать или куда добавить)

In [ ]:
desription(data, 'engineDisplacement')

In [ ]:
# Обработка engineDisplacement
data['engineDisplacement'] = data['engineDisplacement'].apply(
    lambda x: x.replace(' LTR', ''))
data['engineDisplacement'] = data['engineDisplacement'].replace('undefined', 0)

# Перевод в int
data['engineDisplacement'] = data['engineDisplacement'].apply(
    #lambda x: int(float(x)*10))
    lambda x: int(float(x)))

In [ ]:
desription(data, 'engineDisplacement')

Сделали признак числовым, распределение не похоже на нормальное, в дальнейшем будем нормализовывать

In [ ]:
desription(data, 'enginePower')

In [ ]:
data['enginePower'] = data['enginePower'].apply(
    lambda x: int(x.replace(' N12', '')))

In [ ]:
desription(data, 'enginePower')

Сделали признак числовым, распределение не похоже на нормальное, в дальнейшем будем нормализовывать

In [ ]:
desription(data, 'fuelType')

Пропусков нет. Большая часть автомобилей с бензиновым двигателем. Это категориальный признак, поэтому можно использовать его для CatBoost как есть, или сделать One-Hot Encoding для нейросетей. Пригодится для NLP (дальше посмотрим, как его преобразовать или куда добавить)

In [ ]:
desription(data, 'mileage')

распределение не похоже на нормальное, в дальнейшем будем нормализовывать

In [ ]:
desription(data, 'modelDate')

распределение не похоже на нормальное, в дальнейшем будем нормализовывать

In [ ]:
desription(data, 'model_info')

Это категориальный признак, достаточно большое количество уникальных значений. Возможно будем использовать его для CatBoost как есть, или сделать One-Hot Encoding для нейросетей. Или пригодится для NLP (дальше посмотрим, как его преобразовать или куда добавить)

In [ ]:
data.name

Фича дублирует данные о мощности и объеме двигателя, типе коробки и т.д. Возможно попробуем позднее вытащить значимую информацию из признака,а пока удалим

In [ ]:
data = data.drop('name', axis=1)

In [ ]:
desription(data, 'numberOfDoors')

Несмотря на то, что признак имеет числовые значения, по сути он является категориальным, значит, нужно будет сделать преобразование One-Hot Encoding 

In [ ]:
desription(data, 'productionDate')

распределение не похоже на нормальное, в дальнейшем будем нормализовывать

In [ ]:
desription(data, 'vehicleConfiguration')

Фича содержит данные о типе кузова, количестве дверей, типе коробки и объеме двигателя автомобиля.
Данная информация дублируется и в других признаках по отдельности, поэтому удалим его

In [ ]:
data = data.drop('vehicleConfiguration', axis=1)

In [ ]:
desription(data, 'vehicleTransmission')

Это категориальный признак, поэтому можно использовать его для CatBoost как есть, или сделать One-Hot Encoding для нейросетей. Пригодится для NLP (дальше посмотрим, как его преобразовать или куда добавить)

In [ ]:
desription(data, 'Владельцы')

In [ ]:
# там где пропуски считаем, что владелец 1
data['Владельцы'] = data['Владельцы'].fillna(1)

In [ ]:
# Преобразуем в числовой формат
owners_dict = {
    '3 или более': 3,
    '1\xa0владелец': 1,
    '2\xa0владельца': 2
}

data['Владельцы'] = data['Владельцы'].replace(to_replace=owners_dict)

In [ ]:
desription(data, 'Владельцы')

Несмотря на то, что признак имеет числовые значения, по сути он является категориальным, значит, нужно будет сделать преобразование One-Hot Encoding

In [ ]:
desription(data, 'Владение')

Около 75% строк являются пропусками. Удалим данный параметр

In [ ]:
data = data.drop(['Владение'], axis=1)

In [ ]:
desription(data, "ПТС")

Это категориальный признак, поэтому можно использовать его для CatBoost как есть, или сделать One-Hot Encoding для нейросетей. 

In [ ]:
desription(data, "Привод")

Это категориальный признак, поэтому можно использовать его для CatBoost как есть, или сделать One-Hot Encoding для нейросетей. Или пригодится для NLP (дальше посмотрим, как его преобразовать или куда добавить)

In [ ]:
desription(data, "Руль")

всего 2 автомобиля с правым рулем, удалим данный признак, т. к. он не несет полезной информации.

In [ ]:
data = data.drop(['Руль'], axis=1)

In [ ]:
desription(data, "price")

много нулевых значений (это тестовый датасет), и надо будет сделать нормализацию таргета

## Генерация новых признаков

средний пробег в год

In [ ]:
data['mileage_prod_date'] = data['mileage'] / data['productionDate']
data['mileage_prod_date'] = data['mileage_prod_date'].astype('int64')

In [ ]:
desription(data, "mileage_prod_date")

Пробег / кол-во владельцев (сколько владелец в среднем проезжал на авто)

In [ ]:
data['mileage_owns'] = data['mileage'] / data['Владельцы']
data['mileage_owns'] = data['mileage_owns'].astype('int64')

In [ ]:
desription(data, "mileage_owns")

Среднее время владения автомобилем каждым владельцем.

In [ ]:
data['owns_year'] = (2021-data['modelDate']) / data['Владельцы']
data['owns_year'] = data['owns_year'].astype('int64')

In [ ]:
desription(data, "owns_year")

## Значимость фичей

Для оценки значимости числовых переменных будем использовать функцию f_classif из библиотеки sklearn.

Возможности модуля sklearn.feature_selection могут быть использованы не только для выбора важных признаков, но и для уменьшения размерности, улучшения предсказательной силы моделей, либо для повышения их производительности на очень многомерных наборах данных.

В основе метода оценки значимости переменных лежит однофакторный дисперсионный анализ (ANOVA). Основу процедуры составляет обобщение результатов двух выборочных t-тестов для независимых выборок (2-sample t).

В качестве меры значимости мы будем использовать значение f-статистики. Чем значение статистики выше, тем меньше вероятность того, что средние значения не отличаются, и тем важнее данный признак для нашей линейной модели.

In [ ]:
data.info()

In [ ]:
# Категориальные признаки
categorical_features = [
    'bodyType',
    'brand',
    'color',
    'fuelType',
    'model_info',
    'numberOfDoors',
    'vehicleTransmission',
    'Владельцы',
    'ПТС',
    'Привод'
]

# Числовые признаки
numerical_features = [
    'engineDisplacement',
    'mileage',
    'modelDate',
    'mileage_prod_date',
    'mileage_owns',
    'owns_year',
    'productionDate',
    'enginePower'
]

In [ ]:
#проанализируем числовые признаки
#data[numerical_features] - это признаки
#data['price'] - это таргет 
#index = num_cols - а так присваиваются названия признакам, но это именно для сериз
#указываем индекс [0], чтобы выводились значения
imp_num = pd.Series(f_classif(data[numerical_features], data['price'])[
                    0], index=numerical_features)
#сортировка по убыванию
imp_num.sort_values(inplace=True)
#barh - для переворачивания гистограммы
imp_num.plot(kind='barh');

Исходя из значений f-статистики для числовых признаков можно сделать вывод, что productionDate, mileage_prod_date и mileage являются достаточно важными признаками, что вполне логично

In [ ]:
# Функция для проверки статистической разницы в распределении дохода
# с помощью теста Стьюдента встроенного в функцию ttest_ind()
def get_stat_dif(column):
    # На входе имеем имя столбца с нечисловыми (номинативными) данными
    # Проверяем характер распределения оценок в зависимости от значений в этом столбце
    cols = data.loc[:, column].value_counts().index
    # Создаём все возможные парные комбинации из элементов списка cols 
    combinations_all = list(combinations(cols, 2))
    # Для каждой пары номинативных значений рассматриваемого столбца
    # выполняем тест Стьюдента 
    for comb in combinations_all:
        # Готовим аргументы для функции ttest_ind()
        if ttest_ind(data.loc[data.loc[:, column] == comb[0], 'price'], 
                        data.loc[data.loc[:, column] == comb[1], 'price'], nan_policy='omit').pvalue \
            <= 0.05/len(combinations_all): 
            print('Найдены статистически значимые различия для колонки', column)
            break

In [ ]:
for col in categorical_features:
    get_stat_dif(col)

Все категориальные фичи значимы

## Обработка описаний для NLP-модели

In [13]:
data.description

0       Всем привет!\n\nПродам эксклюзивный проект 124...
1       Продаю любимейшую тачилу.. Обвес и тюнинг Hamm...
2       Машина тут продаётся не в первый раз, НО! Толь...
3       Автомобиль в отличном состоянии, весь в родной...
4       Немка! с бесключивым доступом и камерой заднег...
                              ...                        
8348    Продаётся любимый автомобиль! За годы эксплуат...
8349    Куплена у официального дилера в 2017 г. \n\nТр...
8350    Торг на месте. Все работает. Сборка немецкая. ...
8351    Автомобиль в отличном состоянии.\n3-из Авто в ...
8352    Удлиненный заводской Pullman на базе W140 (воо...
Name: description, Length: 8353, dtype: object

* Избавимся от букв латинского алфавита, чисел, знаков препинания и всех символов;
* разобьем пост на токены;
* проведем лемматизацияю, получив нормальную (начальную) форму слова;
* удалим стоп-слова.

In [14]:
patterns = r"[A-Za-z0-9^\W]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()
def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    return tokens

In [15]:
data.description = data.description.apply(lemmatize)

In [16]:
data.description

0       [весь, привет, продать, эксклюзивный, проект, ...
1       [продавать, любимый, тачил, обвес, тюнинг, пот...
2       [машина, продаваться, первый, но, только, пере...
3       [автомобиль, отличный, состояние, весь, родный...
4       [немка, бесключивый, доступ, камера, задний, в...
                              ...                        
8348    [продаваться, любимый, автомобиль, за, год, эк...
8349    [купить, официальный, дилер, г, третий, владел...
8350    [торг, место, всё, работать, сборка, немецкий,...
8351    [автомобиль, отличный, состояние, авто, семья,...
8352    [удлинённый, заводский, база, вообще, правильн...
Name: description, Length: 8353, dtype: object

Посмотрим самые распространенные слова в описании

In [17]:
word_freq = defaultdict(int)
for tokens in data.description.dropna():
    for token in tokens:
        word_freq[token] += 1

In [18]:
len(word_freq)

13617

In [19]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['автомобиль',
 'система',
 'пробег',
 'задний',
 'салон',
 'сидение',
 'год',
 'состояние',
 'передний',
 'машина']

In [20]:
# TOKENIZER
# The maximum number of words to be used. (most frequent)
MAX_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 256

In [21]:
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(data.description)

In [22]:
tokenize.word_index

{'автомобиль': 1,
 'система': 2,
 'пробег': 3,
 'задний': 4,
 'салон': 5,
 'сидение': 6,
 'год': 7,
 'состояние': 8,
 'передний': 9,
 'машина': 10,
 'диск': 11,
 'комплект': 12,
 'в': 13,
 'дилер': 14,
 'новый': 15,
 'официальный': 16,
 'ваш': 17,
 'кредит': 18,
 'фара': 19,
 'авто': 20,
 'гарантия': 21,
 'пакет': 22,
 'руль': 23,
 'всё': 24,
 'колесо': 25,
 'безопасность': 26,
 'комплектация': 27,
 'вы': 28,
 'обмен': 29,
 'птс': 30,
 'контроль': 31,
 'зеркало': 32,
 'проверка': 33,
 'покупка': 34,
 'продажа': 35,
 'водитель': 36,
 'полный': 37,
 'шина': 38,
 'датчик': 39,
 'км': 40,
 'отличный': 41,
 'пассажир': 42,
 'двигатель': 43,
 'резина': 44,
 'условие': 45,
 'владелец': 46,
 'ключ': 47,
 'то': 48,
 'кузов': 49,
 'подушка': 50,
 'подогрев': 51,
 'обогрев': 52,
 'технический': 53,
 'любой': 54,
 'обслуживание': 55,
 'хороший': 56,
 'два': 57,
 'юридический': 58,
 'оригинал': 59,
 'стекло': 60,
 'торг': 61,
 'масло': 62,
 'м': 63,
 'дверь': 64,
 'замена': 65,
 'чёрный': 66,
 'про

In [24]:
sub_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(data.description), maxlen=MAX_SEQUENCE_LENGTH)

print(sub_sequences)

[[   0    0    0 ...  133  643  218]
 [   0    0    0 ... 7868  820 2149]
 [   0    0    0 ...    8   61  370]
 ...
 [   0    0    0 ...  635 1737  752]
 [   0    0    0 ...  852 1679 1462]
 [   0    0    0 ...  825   72    3]]


In [25]:
# вот так теперь выглядит наш текст
print(data.iloc[6])
print(sub_sequences[6])

bodyType                                                внедорожник 5 дв.
brand                                                                AUDI
color                                                          коричневый
description             [внимание, только, клиент, автомобиль, пробег,...
engineDisplacement                                                3.0 LTR
enginePower                                                       245 N12
fuelType                                                           дизель
mileage                                                            133615
modelDate                                                            2009
model_info                                                             Q7
name                                               3.0d AT (245 л.с.) 4WD
numberOfDoors                                                           5
productionDate                                                       2013
sell_id                               

In [ ]:
data.to_csv('data.csv', index=False)

# PreProc Tabular Data

In [ ]:
#пока такая предобработка, не трогаем категориальные фичи  и не заполняем пропуски 
def preproc_data(df_input):
    
    df_output = df_input.copy()
    
    # убираем не нужные для модели признаки
    df_output.drop(['description', 'sell_id'], axis = 1, inplace=True)

    # Логарифмирование
    for column in numerical_features:
        df_output[column] = np.log(df_output[column] + 1)
        #логарифмируем и таргет тоже
    df_output['price'] = np.log(df_output['price'] + 1)
    
    # Нормализация данных
    scaler = MinMaxScaler()
    for column in numerical_features:
        df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
        
    return df_output

In [ ]:
# Запускаем и проверяем, что получилось
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [ ]:
df_preproc.info()

## Split data

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.price.values     # наш таргет
X = train_data.drop(['price'], axis=1)
X_sub = test_data.drop(['price'], axis=1) # на этой части будем тестировать

In [ ]:
test_data.info()

# Model 2: CatBoostRegressor

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
model = CatBoostRegressor(iterations = 5000, #8000
                          #depth=10,
                          #learning_rate = 0.5,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['RMSE', 'MAE'],
                          od_wait=500,
                          #task_type='GPU',
                         )
model.fit(X_train, y_train,
         eval_set=(X_test, y_test),
          cat_features=categorical_features,
         verbose_eval=100,
         use_best_model=True,
         #plot=True
         )

In [ ]:
test_predict_catboost = model.predict(X_test)
print(f"TEST mape: {(mape(y_test, test_predict_catboost))*100:0.2f}%")

### Submission

In [ ]:
sub_predict_catboost = model.predict(X_sub)
#мы прологарифмировали таргет, поэтому надо сделать обратную процедуру
sub_predict_catboost = np.exp(sub_predict_catboost)

sample_submission['price'] = sub_predict_catboost
sample_submission.to_csv('without_round_catboost.csv', index=False)

# округлим
sub_predict_catboost = (sub_predict_catboost // 1000) * 1000

sample_submission['price'] = sub_predict_catboost
sample_submission.to_csv('catboost_submission.csv', index=False)

### подбор ГП для Catboost

In [ ]:
#определим модель
model = CatBoostRegressor(iterations=1000)

In [ ]:
train_pool = Pool(X_train, y_train, cat_features=categorical_features)
test_pool = Pool(X_test, cat_features=categorical_features)

In [ ]:
#подберем параметры с помощью сетки (долго считается без GPU, порядка 3-4 часов, не успела до дедлайна)
grid = {
    'learning_rate': [0.03, 0.1],
    'depth':[4, 6, 8, 10, 12],
    'l2_leaf_reg': [0,5, 1, 3, 5, 7, 9],
    'iterations': [1000, 2000, 3000, 4000, 5000, 6000],
    'bootstrap_type': ['Bayesian', 'Bernoulli', 'MVS', None],
    #'eval_metric': ['MAPE'],
    'od_type': ['Iter', 'IncToDec']  
}
grid_search_results = model.grid_search(grid, train_pool, shuffle=False, verbose=True, plot=True)

In [ ]:
#параметры для лучшего значения
grid_search_results['params']

In [ ]:
#посмотрим на все параметры модели
model.get_all_params()

In [ ]:
test_predict_catboost = model.predict(X_test)
print(f"TEST mape: {(mape(y_test, test_predict_catboost))*100:0.2f}%")

In [ ]:
sub_predict_catboost = model.predict(X_sub)
#мы прологарифмировали таргет, поэтому надо сделать обратную процедуру
sub_predict_catboost = np.exp(sub_predict_catboost)

sample_submission['price'] = sub_predict_catboost
sample_submission.to_csv('without_round_catboost_ft.csv', index=False)

# округлим
sub_predict_catboost = (sub_predict_catboost // 1000) * 1000

sample_submission['price'] = sub_predict_catboost
sample_submission.to_csv('catboost_submission_ft.csv', index=False)

__Выводы по части 1:__
* Построена "наивная" модель для задания начального ориентира по метрики.
* Точность "наивной" модели по метрике MAPE: 19.88%.
* Проведен анализ и обработка признаков.
* Сгенерированы новые признаки (feature engineering).
* Проведена оценка значимости признаков.
* Обработаны текстовые описания для передачи в NLP-модель (очистка, нормализация, лемматизация, удаление стоп-слов).
* Проведена нормализация и логарифмирование числовых признаков и таргета.
* Протестирована и модель CatBoostRegressor (подбор гиперпараметров ОЧЕНЬ долго считается)
* Точность CatBoostRegressor модели по метрике MAPE: 12.24% (прирост 0,01 достигнут с помощью огругления таргета)

### albumentations